In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.MachineModels import MachineModels 

import numpy as np
import datetime
import pandas as pd
import polars as pl
import optuna

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-lstm-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-08-20 18:45:02,107 - This will print to the notebook's output cell


c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Default params (keeps previous behavior where params corresponds to idxAfterPrediction=5)
params_default = {
    "idxAfterPrediction": 5,
    'timesteps': 90,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 3,
    "LSTM_dropout": 0.001,
    "LSTM_recurrent_dropout": 0.001,
    "LSTM_learning_rate": 0.001,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 10,
    "LSTM_l1": 0.001,
    "LSTM_l2": 0.001,
    "LSTM_inter_dropout": 0.001,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",
}

In [ ]:
stock_group = "group_regOHLCV_over5years"
eval_date = datetime.date(year=2025, month=6, day=13)
split_date = datetime.date(year=2023, month=12, day=31)

studytime = 60*60*4
studyname = f"sandbox_lstm_optuna_{formatted_str}"
n_startup_trials = 15
q_full = 0.99

In [ ]:
def objective(trial: optuna.Trial) -> float:
    opt_params = params_default.copy()
    opt_params["year_start"] = trial.suggest_int("year_start", 2019, 2023)
    opt_params["idxAfterPrediction"] = trial.suggest_int("idxAfterPrediction", 3, 8, step=1)
    opt_params["LoadupSamples_time_inc_factor"] = trial.suggest_int("LoadupSamples_time_inc_factor", 21, 81, step=5)
    opt_params["timesteps"] = trial.suggest_int("timesteps", 30, 90, step=5)
    opt_params["LSTM_units"] = 16
    opt_params["LSTM_num_layers"] = 1
    opt_params["LSTM_learning_rate"] = trial.suggest_float("LSTM_learning_rate", 1e-5, 1e-3, log=True)
    opt_params["LSTM_epochs"] = 2
    opt_params["LSTM_l1"] = trial.suggest_float("LSTM_l1", 1e-5, 1e-3, log=True)
    opt_params["LSTM_l2"] = trial.suggest_float("LSTM_l2", 1e-4, 1e-1, log=True)
    opt_params["LSTM_dropout"] = trial.suggest_float("LSTM_dropout", 1e-4, 1e-2, log=True)
    opt_params["LSTM_inter_dropout"] = trial.suggest_float("LSTM_inter_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_recurrent_dropout"] = trial.suggest_float("LSTM_recurrent_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_conv1d_kernel_size"] = 3
    opt_params["is_single_feature"] = trial.suggest_categorical("is_single_feature", [True, False])

    ls = LoadupSamples(
        train_start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        test_dates=[eval_date],
        treegroup=None,
        timegroup=stock_group,
        params=opt_params,
    )
    ls.load_samples(main_path = "../src/featureAlchemy/bin/")

    ls.split_dataset(
        start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        last_train_date=split_date,
        last_test_date=eval_date,
    )

    Xtrain = ls.train_Xtime
    ytrain = ls.train_ytime
    Xtest = ls.test_Xtime
    ytest = ls.test_ytime

    true_res = ls.meta_pl_test

    Xtrain = Xtrain[:, -opt_params["timesteps"]:, :]
    Xtest = Xtest[:, -opt_params["timesteps"]:, :]

    if opt_params["is_single_feature"]:
        Xtrain = Xtrain[:, :, [0]]
        Xtest = Xtest[:, :, [0]]

    mm = MachineModels(opt_params)

    starttime0 = datetime.datetime.now()
    model_lstm0, res_dict0 = mm.run_LSTM_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
    preds_test0 = mm.predict_LSTM_torch(model_lstm0, Xtest, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    endtime0 = datetime.datetime.now()

    # final mask
    q0 = q_full
    mask_pred0_test_above = (np.abs(preds_test0) >= np.quantile(np.abs(preds_test0),   q0))
    mask_pred0_test_below = (np.abs(preds_test0) <= np.quantile(np.abs(preds_test0), 1-q0))

    true_res_masked_above = true_res.filter(pl.Series(mask_pred0_test_above))
    true_res_masked_below = true_res.filter(pl.Series(mask_pred0_test_below))

    score = (np.mean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/opt_params["idxAfterPrediction"])

    # Log some results
    def quant_dis_in_mask(mask: np.ndarray, q: float) -> float:
        if not mask.any():
            return len(mask)
        return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)

    fullmask_above = mask_pred0_test_above.copy()
    fullmask_below = mask_pred0_test_below.copy()

    logger.info(f"Trial {trial.number} with params: {opt_params}")
    logger.info(f"  q0: {q0:.4f}")
    logger.info(f"  Duration0: {endtime0 - starttime0}")
    logger.info(f"  Val RMSE0 adjusted: {res_dict0['val_rmse']/opt_params['LoadupSamples_time_inc_factor']:.4f}")
    logger.info(f"  Mean all prediction: {np.mean(true_res['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean above prediction: {np.mean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean below prediction: {np.mean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Quantile 0.99 for distance in mask above: {quant_dis_in_mask(fullmask_above, 0.99)}")
    logger.info(f"  Quantile 0.99 for distance in mask below: {quant_dis_in_mask(fullmask_below, 0.99)}")
    logger.info(f"  Ratio for quantile-distance-to-length above: {quant_dis_in_mask(fullmask_above, 0.99) / len(fullmask_above):.4f}")
    logger.info(f"  Ratio for quantile-distance-to-length below: {quant_dis_in_mask(fullmask_below, 0.99) / len(fullmask_below):.4f}")
    logger.info(f"  Score: {score:.4f}")

    return score

optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=n_startup_trials)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(objective, timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-08-20 18:45:02,511] A new study created in RDB with name: sandbox_lstm_optuna_notebook-lstm-optuna-20aug25_1845


2025-08-20 18:45:02,511 - A new study created in RDB with name: sandbox_lstm_optuna_notebook-lstm-optuna-20aug25_1845
2025-08-20 18:45:07,034 - Non-finite/too-large values in train y time: 84 samples.
2025-08-20 18:45:07,039 - Removing 84 samples from training data.


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-20 18:45:22,265 - Epoch 1/2 — Train RMSE: 0.5229 — Validation RMSE: 0.4340


Epochs:  50%|█████     | 1/2 [00:10<00:10, 10.51s/it]

...

2025-08-20 22:45:16,928 - Epoch 2/2 — Train RMSE: 0.4732 — Validation RMSE: 0.4192


Epochs: 100%|██████████| 2/2 [00:31<00:00, 15.76s/it]


2025-08-20 22:45:20,851 - Trial 220 with params: {'idxAfterPrediction': 3, 'timesteps': 90, 'target_option': 'last', 'LoadupSamples_time_scaling_stretch': True, 'LoadupSamples_time_inc_factor': 71, 'LSTM_units': 16, 'LSTM_num_layers': 1, 'LSTM_dropout': 0.0046837734431996745, 'LSTM_recurrent_dropout': 0.001, 'LSTM_learning_rate': 7.934005843436357e-05, 'LSTM_optimizer': 'adam', 'LSTM_bidirectional': True, 'LSTM_batch_size': 4096, 'LSTM_epochs': 2, 'LSTM_l1': 0.001, 'LSTM_l2': 0.001, 'LSTM_inter_dropout': 0.001, 'LSTM_input_gaussian_noise': 0.001, 'LSTM_conv1d': True, 'LSTM_conv1d_kernel_size': 3, 'LSTM_loss': 'mse', 'year_start': 2022, 'is_single_feature': False}
2025-08-20 22:45:20,851 -   q0: 0.9980
2025-08-20 22:45:20,853 -   Duration0: 0:00:35.555595
2025-08-20 22:45:20,853 -   Val RMSE0 adjusted: 0.0059
2025-08-20 22:45:20,853 -   Mean all prediction: 1.0007
2025-08-20 22:45:20,853 -   Mean above prediction: 0.9967
2025-08-20 22:45:20,853 -   Mean below prediction: 0.9943
2025-08-

[I 2025-08-20 22:45:20,901] Trial 220 finished with value: 0.9989120168718092 and parameters: {'year_start': 2022, 'idxAfterPrediction': 3, 'LoadupSamples_time_inc_factor': 71, 'timesteps': 90, 'LSTM_learning_rate': 7.934005843436357e-05, 'LSTM_dropout': 0.0046837734431996745, 'is_single_feature': False}. Best is trial 31 with value: 1.0049487769776475.


2025-08-20 22:45:20,901 - Trial 220 finished with value: 0.9989120168718092 and parameters: {'year_start': 2022, 'idxAfterPrediction': 3, 'LoadupSamples_time_inc_factor': 71, 'timesteps': 90, 'LSTM_learning_rate': 7.934005843436357e-05, 'LSTM_dropout': 0.0046837734431996745, 'is_single_feature': False}. Best is trial 31 with value: 1.0049487769776475.
2025-08-20 22:45:20,901 - Best parameters: {'year_start': 2020, 'idxAfterPrediction': 4, 'LoadupSamples_time_inc_factor': 61, 'timesteps': 30, 'LSTM_learning_rate': 0.00010773374852638822, 'LSTM_dropout': 0.021106707856634717, 'is_single_feature': False}
2025-08-20 22:45:20,901 - Best score: 1.0049487769776475
2025-08-20 22:45:20,917 - 
Trials DataFrame:
2025-08-20 22:45:20,917 -      number     value             datetime_start          datetime_complete               duration  params_LSTM_dropout  params_LSTM_learning_rate  params_LoadupSamples_time_inc_factor  params_idxAfterPrediction  params_is_single_feature  params_timesteps  params

In [5]:
df.to_parquet(f"sandbox_lstm_optuna_{formatted_str}.parquet", index=False)

In [6]:
# Run with best parameters
best_params = {**params_default, **study.best_params.copy()}
best_params["LSTM_units"] = 16
best_params["LSTM_epochs"] = 2
best_params["LSTM_conv1d_kernel_size"] = 3
best_params["LSTM_num_layers"] = 1

ls = LoadupSamples(
    train_start_date=datetime.date(year=best_params["year_start"], month=1, day=1),
    test_dates=[eval_date],
    group=stock_group,
    group_type="Time",
    params=best_params,
)
ls.load_samples(main_path="../src/featureAlchemy/bin/")

ls.split_dataset(
    start_date=datetime.date(year=best_params["year_start"], month=1, day=1),
    last_train_date=split_date,
    last_test_date=eval_date,
)

Xtrain = ls.train_Xtime
ytrain = ls.train_ytime
Xtest = ls.test_Xtime
ytest = ls.test_ytime

Xtrain = Xtrain[:, -best_params["timesteps"]:, :]
Xtest = Xtest[:, -best_params["timesteps"]:, :]

if best_params["is_single_feature"]:
    Xtrain = Xtrain[:, :, [0]]
    Xtest = Xtest[:, :, [0]]

mm = MachineModels(best_params)
model_lstm, res_dict = mm.run_LSTM_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
preds = mm.predict_LSTM_torch(model_lstm, Xtest, batch_size=best_params["LSTM_batch_size"], device="cuda")



TypeError: LoadupSamples.__init__() got an unexpected keyword argument 'group'

In [ ]:
q = 0.98
mask_pred_above = (preds >= np.quantile(preds, q))
mask_pred_below = (preds <= np.quantile(preds, 1-q))
true_res = ls.meta_pl_test
true_res_masked_above = true_res.filter(pl.Series(mask_pred_above))
true_res_masked_below = true_res.filter(pl.Series(mask_pred_below))
score = (np.mean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/best_params["idxAfterPrediction"])
# Log some results
def quant_dis_in_mask(mask: np.ndarray, q: float) -> int:
    if not mask.any():
        return len(mask)
    return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)
logger.info(f"  Val RSME adjusted: {res_dict['val_rmse']/best_params['LoadupSamples_time_inc_factor']:.4f}")
logger.info(f"  Mean all prediction: {np.mean(true_res['target_ratio'].to_numpy()):.4f}")
logger.info(f"  Mean above prediction: {np.mean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
logger.info(f"  Mean below prediction: {np.mean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
logger.info(f"  Quantile 0.99 in mask above: {quant_dis_in_mask(mask_pred_above, 0.99)}")
logger.info(f"  Quantile 0.99 in mask below: {quant_dis_in_mask(mask_pred_below, 0.99)}")
logger.info(f"  Ratio for quantile to length above: {quant_dis_in_mask(mask_pred_above, 0.99) / len(mask_pred_above):.4f}")
logger.info(f"  Ratio for quantile to length below: {quant_dis_in_mask(mask_pred_below, 0.99) / len(mask_pred_below):.4f}")
logger.info(f"  Score: {score:.4f}")

2025-08-16 18:33:38,602 -   Val RSME adjusted: 0.0132
2025-08-16 18:33:38,604 -   Mean all prediction: 1.0010
2025-08-16 18:33:38,605 -   Mean above prediction: 1.0101
2025-08-16 18:33:38,605 -   Mean below prediction: 1.0001
2025-08-16 18:33:38,608 -   Quantile 0.99 in mask above: 450.27999999999884
2025-08-16 18:33:38,609 -   Quantile 0.99 in mask below: 427.0
2025-08-16 18:33:38,611 -   Ratio for quantile to length above: 0.0007
2025-08-16 18:33:38,612 -   Ratio for quantile to length below: 0.0006
2025-08-16 18:33:38,612 -   Score: 1.0025
